<a href="https://colab.research.google.com/github/GyamfiKodie/Lane_Detection/blob/main/Copy_of_TUSimple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import numpy as np
import json
import cv2
import os
from glob import glob
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import json
import cv2
import numpy as np

DATASET_PATH = "/content/drive/MyDrive/Machine_Learning/Tusimple/train_set"
SAVE_PATH = "/content/drive/MyDrive/Machine_Learning/Tusimple/processed_data"  # Directory to save processed data

# Ensure the save directory exists
os.makedirs(SAVE_PATH, exist_ok=True)

# Function to load label data
def load_labels(label_file):
    with open(label_file, 'r') as f:
        return [json.loads(line) for line in f]

# Load all label data
# Load all label data
labels_0313 = load_labels(os.path.join(DATASET_PATH, '/content/drive/MyDrive/Machine_Learning/Tusimple/train_set/label_data_0313.json'))
labels_0531 = load_labels(os.path.join(DATASET_PATH, '/content/drive/MyDrive/Machine_Learning/Tusimple/train_set/label_data_0531.json'))
labels_0601 = load_labels(os.path.join(DATASET_PATH, '/content/drive/MyDrive/Machine_Learning/Tusimple/train_set/label_data_0601.json'))

all_labels = labels_0313 + labels_0531 + labels_0601

# Function to load images
def load_images(label_data):
    images = []
    for item in label_data:
        img_path = os.path.join(DATASET_PATH, item['raw_file'])
        img = cv2.imread(img_path)
        img = cv2.resize(img, (320, 160)) # Resize to a manageable size
        images.append(img)
    return np.array(images)

# Load images
images = load_images(all_labels)

# Function to create lane masks
def create_lane_masks(label_data):
    masks = []
    for item in label_data:
        mask = np.zeros((720, 1280)) # Assuming original size
        lanes = item['lanes']
        h_samples = item['h_samples']
        for lane in lanes:
            if lane.count(-2) < len(lane):  # if not all points are -2
                for x, y in zip(lane, h_samples):
                    if x != -2:
                        cv2.circle(mask, (x, y), 5, 255, -1)  # Draw circles on lane points
        mask = cv2.resize(mask, (320, 160))  # Resize to match image size
        masks.append(mask)
    return np.array(masks)

# Create lane masks
masks = create_lane_masks(all_labels)

# Normalize images
images = images / 255.0
masks = masks / 255.0

# Expand dimensions for masks to match the shape required by the model
masks = np.expand_dims(masks, axis=-1)

# Save the processed images and masks
np.save(os.path.join(SAVE_PATH, 'images.npy'), images)
np.save(os.path.join(SAVE_PATH, 'masks.npy'), masks)

print("Processed data saved successfully!")



Processed data saved successfully!


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

# Define the paths to the saved .npy files
SAVE_PATH = "/content/drive/MyDrive/Machine_Learning/Tusimple/processed_data"
IMAGES_PATH = os.path.join(SAVE_PATH, 'images.npy')
MASKS_PATH = os.path.join(SAVE_PATH, 'masks.npy')

# Load the processed images and masks
images = np.load(IMAGES_PATH)
masks = np.load(MASKS_PATH)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, masks, test_size=0.2, random_state=42)

# Print the shapes of the resulting arrays
print(f"Training images shape: {X_train.shape}")
print(f"Training masks shape: {y_train.shape}")
print(f"Validation images shape: {X_val.shape}")
print(f"Validation masks shape: {y_val.shape}")

# Save the split data for future use
np.save(os.path.join(SAVE_PATH, 'X_train.npy'), X_train)
np.save(os.path.join(SAVE_PATH, 'y_train.npy'), y_train)
np.save(os.path.join(SAVE_PATH, 'X_val.npy'), X_val)
np.save(os.path.join(SAVE_PATH, 'y_val.npy'), y_val)

print("Data split and saved successfully!")


In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score
import os

# Define the paths to the saved .npy files
SAVE_PATH = "/content/drive/MyDrive/Machine_Learning/Tusimple/train_val"
X_TRAIN_PATH = os.path.join(SAVE_PATH, 'X_train.npy')
Y_TRAIN_PATH = os.path.join(SAVE_PATH, 'y_train.npy')
X_VAL_PATH = os.path.join(SAVE_PATH, 'X_val.npy')
Y_VAL_PATH = os.path.join(SAVE_PATH, 'y_val.npy')

# Load the training and validation data
X_train = np.load(X_TRAIN_PATH)
y_train = np.load(Y_TRAIN_PATH)
X_val = np.load(X_VAL_PATH)
y_val = np.load(Y_VAL_PATH)

image_height = 160
image_width = 320

# Define your model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(image_height, image_width, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(1, (3, 3), activation='sigmoid', padding='same'))

# Compile the model
model.compile(optimizer=Adam(lr=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X_val, y_val)
print("Validation Loss:", loss)
print("Validation Accuracy:", accuracy)

